In [1]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
%matplotlib inline

import pathlib
from pathlib import Path
import matplotlib.pyplot as plot
import librosa
import tensorflow as tf
import keras
from keras import regularizers
from keras.models import load_model, Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
    
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from numpy import argmax
from imutils import paths
import numpy as np

import random
from datetime import datetime
import pickle
import os
import sys
import json
import re

#Visualization
import seaborn as sn
import pandas as pd

import warnings

#env parameters
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
#Paramters
data_folder = Path("../../../audio/testfiles/GTZAN_test/genres/")
output_root = './../../../models/'
output_folder = Path('./output/cvnn.model')
output_model = output_root + '/cnn_dong_model_weights.h5'
output_architecture = output_root + '/cnn_dong_model_architecture.json'
output_whole = output_root + 'cnn_dong_model_whole.h5'
output_best_model = output_root + 'best_model.h5'
output_label = output_root + 'label.pkl'
output_test_paths = output_root + 'test_paths.pkl'
spectogram_folder = Path("./img_data/")
# Duration of songsnippet in seconds
duration = 2.97
start_offset = 0
epochs = 4
num_segments = 1

batch_size = 128

es_patience = 20
return_train_and_test = 1

sr = 22050 # Sampling rate

#Parameters for mel spec
fmax = 1500 # maximum frequency considered
fft_points = 512
fft_dur = fft_points * 1.0 / sr # 23ms windows
hop_size = int(fft_points/ 2) # 50% overlap between consecutive frames
n_mels = 64

#Segment duration
num_fft_windows = 256 #per Segment
segment_in_points = num_fft_windows * 255
segment_dur = segment_in_points * 1.0 / sr

input_shape=(64, 256, 1)

randomseed = 11
#randomseed = datetime.now()
# Seed for RNG
random.seed(randomseed)


In [3]:
#Get directories of all songs
songs = []
genres = []

for g in data_folder.iterdir():
    genres.append(g.name)
    for i in g.iterdir():
        songs.append(i)

In [4]:
mel_specs = []  
labels = []

#loading with different numbers of segements for data-augmentation
#!Initialize "spectograms" before running
def load_specs(data = songs, num_segments = 1):
    spectograms = []
    for song in data:
        offset = start_offset
        for i in range(num_segments):
            y, sr = librosa.load(song, mono=True, offset=offset, duration=duration)
            m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_points,
                                                  hop_length=hop_size, n_mels=n_mels,
                                                  fmax=fmax)
            mel_specs.append([m_sp])
            label = song.parts[-2]
            labels.append(label)
            spectograms.append( (m_sp, label, song, offset, duration) )
            offset = offset + duration/2
        input_shape = m_sp.shape + (1,)
    return spectograms

In [5]:
#shuffle songs to keep segements together
random.shuffle(songs)

In [6]:
spectograms = load_specs(data = songs, num_segments = num_segments)
len(spectograms)

60

In [7]:
print("Total number of samples: ", len(mel_specs))

#set dynamic input shape
input_shape = np.shape(mel_specs[2]) + (1,)
input_shape = (input_shape[1], input_shape[2], input_shape[3])
print("Input shape: " + str(input_shape))

Total number of samples:  60
Input shape: (64, 256, 1)


In [8]:
#Split into Train and Testing
testsplit = len(spectograms)*0.7    #70% train-test-split
train = spectograms[:int(testsplit)]
test = spectograms[int(testsplit):]

x_train, y_train, p_train, offset_train, duration_train = zip(*train)
x_test, y_test, p_test, offset_test, duration_test = zip(*test)

#Fit Dimensions
x_train = np.array([x.reshape(input_shape) for x in x_train])
x_test = np.array([x.reshape(input_shape) for x in x_test])

In [9]:
#Binarize Labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [10]:
def dump_testspecs_labels():
    with open(output_label, 'wb') as f:
        pickle.dump(lb, f)

    if return_train_and_test == 1:
        r_paths = p_test + (p_train)
        r_offsets = offset_test + (offset_train)
        r_durations = duration_test + (duration_train)
    else:
        r_paths = p_test
        r_offsets = offset_test
        r_durations = duration_test

    r_values = [r_paths, r_offsets, r_durations]

    with open(output_test_paths, 'wb') as f:
        pickle.dump(r_values, f)

In [11]:
#dump_testspecs_labels():

In [12]:
#old model for urban
def cnn_urban_model_build():
    model = Sequential()

    model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
    model.add(MaxPooling2D((4, 2), strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, (5, 5), padding="valid"))
    model.add(MaxPooling2D((4, 2), strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, (5, 5), padding="valid"))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dropout(rate=0.5))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.5))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(
        optimizer="Adam",
        loss="categorical_crossentropy",
        metrics=['accuracy'])
    print(model.summary)
    return model

#model for GTZAN
def cnn_dong_model_build():

    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3),
                     activation='relu', kernel_regularizer=regularizers.l2(0.01),
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 4)))

    model.add(Conv2D(64, (3, 5), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(MaxPooling2D(pool_size=(2, 4)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
    model.add(Dropout(0.3))
    model.add(Dense(len(genres), activation='softmax'))
    #model.compile(loss=keras.losses.categorical_crossentropy,
     #             optimizer=keras.optimizers.Adadelta(decay=1e-6),
      #            metrics=['accuracy'])
    model.compile(
        optimizer="Adam",
        loss="categorical_crossentropy",
        metrics=['accuracy'])
    print(model.summary)
    return model

In [13]:
def train_model():
    model = cnn_dong_model_build()

    callbacks = [EarlyStopping(monitor='val_loss', patience=es_patience),
                 ModelCheckpoint(filepath=output_best_model, monitor='val_loss', save_best_only=True)]

    H = model.fit(
        x=x_train, 
        y=y_train,
        epochs=epochs,
        callbacks=callbacks,
        batch_size=batch_size,
        validation_data= (x_test, y_test))

    score = model.evaluate(x=x_test,y=y_test)

    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return model

In [14]:
def save_model():
    # Save the weights
    model.save_weights(output_model)

    # Save the model architecture
    with open(output_architecture, 'w') as f:
        f.write(model.to_json())

    # Save complete model
    model.save(output_whole)

In [15]:
model = train_model()

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7f27a8460e80>>
Train on 42 samples, validate on 18 samples
Epoch 1/4
42/42 [==============================] - 2s 43ms/step - loss: 5.5693 - acc: 0.0952 - val_loss: 7.7844 - val_acc: 0.1111
Epoch 2/4
42/42 [==============================] - 1s 32ms/step - loss: 5.8243 - acc: 0.1905 - val_loss: 5.3501 - val_acc: 0.1667
Epoch 3/4
42/42 [==============================] - 1s 34ms/step - loss: 4.4031 - acc: 0.3333 - val_loss: 4.5545 - val_acc: 0.1111
Epoch 4/4
18/18 [==============================] - 0s 7ms/step
Test loss: 4.163966178894043
Test accuracy: 0.1111111119389534


In [16]:
save_model()

In [17]:
def evaluate_song(song):
    start = 0
    r = 0
    for i in range(int(30/(duration/2))-1):
        y, sr = librosa.load(song, mono=True, offset=start, duration=duration)
        m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_points,
                                              hop_length=hop_size, n_mels=n_mels,
                                              fmax=fmax)
        m_sp = np.expand_dims(m_sp, 0)
        m_sp = np.expand_dims(m_sp, 3)
        if r == 0:
            prediction = model.predict(m_sp)
            r = 1
        else:
            prediction = (prediction + model.predict(m_sp))/2
        start = start + duration/2
    return prediction
        
def evaluate_batch(batch):
    predictions = []
    for song in batch:
        predictions.append(evaluate_song(song))
    return predictions

def compare_batch(predictions):
    i = 0
    l = lb.inverse_transform(y_test)
    counter = 0
    for prediction in predictions:
        tr_value = str(l[i])
        pr_value = re.sub('[\[\]\']', '', str(lb.classes_[prediction.argmax(axis=1)]))
        print(tr_value + "=>" + pr_value)
        if tr_value == pr_value:
            counter += 1
        i += 1
    acc = counter/len(predictions)*100
    print("Acc: " + str(acc) + "%")

In [18]:
model = load_model(output_whole)

In [19]:
predictions = evaluate_batch(p_test)
print(predictions)

[array([[0.10733238, 0.05372522, 0.15000305, 0.05761873, 0.10228785,
        0.16732156, 0.09410034, 0.12233038, 0.07740969, 0.06787084]],
      dtype=float32), array([[0.11593819, 0.02786388, 0.11218724, 0.042406  , 0.10997602,
        0.2286258 , 0.08105122, 0.13740441, 0.06754923, 0.07699804]],
      dtype=float32), array([[3.6127508e-01, 3.7802660e-05, 6.3936867e-02, 1.6394507e-03,
        1.0107747e-02, 3.2318134e-02, 6.1927749e-05, 1.5011637e-02,
        3.0346238e-03, 5.1257670e-01]], dtype=float32), array([[0.08470407, 0.0197611 , 0.17953536, 0.03227285, 0.22265741,
        0.24924105, 0.05443884, 0.08135629, 0.04281237, 0.0332206 ]],
      dtype=float32), array([[0.07939922, 0.04697099, 0.14452475, 0.0547954 , 0.13107175,
        0.20238805, 0.0879993 , 0.1391128 , 0.05598417, 0.05775359]],
      dtype=float32), array([[0.11105043, 0.08569268, 0.10562597, 0.09016043, 0.0910494 ,
        0.11209818, 0.0867638 , 0.10922344, 0.09975161, 0.10858406]],
      dtype=float32), array([

In [20]:
compare_batch(predictions)

metal=>jazz
disco=>jazz
hiphop=>rock
pop=>jazz
pop=>jazz
classical=>jazz
metal=>jazz
disco=>jazz
country=>jazz
pop=>jazz
classical=>jazz
hiphop=>blues
jazz=>country
metal=>jazz
hiphop=>jazz
classical=>jazz
hiphop=>jazz
jazz=>jazz
Acc: 5.555555555555555%


In [21]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x_test, batch_size=batch_size)




 
# plot the training loss and accuracy
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
"""
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
"""
0
#print(predictions)

[INFO] evaluating network...


0

<Figure size 432x288 with 0 Axes>

In [22]:
print(classification_report(y_test.argmax(axis=1),
                            predictions.argmax(axis=1), 
                            target_names=lb.classes_))

ValueError: Number of classes, 8, does not match size of target_names, 10. Try specifying the labels parameter

In [ ]:
cm = confusion_matrix(lb.inverse_transform(y_test), lb.classes_[predictions.argmax(axis=1)], genres)

cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm = np.around(cm, decimals=2)
print(genres)
print(cm)
print(lb.classes_[predictions.argmax(axis=1)])



In [ ]:
p_array = [[0.34,0.03,0.23,0.08,0.02,0.03,0.02,0.09,0.16,0.01],
 [0.02, 0.68, 0.02, 0.1 , 0.  , 0.01, 0.04, 0.02, 0.02, 0.08],
 [0.09, 0.04, 0.63, 0.03, 0.07, 0.02, 0.01, 0.1 , 0.01, 0.  ],
 [0.  , 0.03, 0.06, 0.67, 0.01, 0.02, 0.05, 0.12, 0.  , 0.04],
 [0.02, 0.  , 0.06, 0.02, 0.61, 0.1 , 0.01, 0.15, 0.02, 0.  ],
 [0.  , 0.  , 0.01, 0.  , 0.02, 0.97, 0.  , 0.  , 0.  , 0.  ],
 [0.02, 0.04, 0.03, 0.03, 0.03, 0.04, 0.58, 0.07, 0.02, 0.15],
 [0.09, 0.  , 0.06, 0.02, 0.07, 0.02, 0.02, 0.63, 0.09, 0.  ],
 [0.17, 0.03, 0.01, 0.01, 0.  , 0.  , 0.01, 0.04, 0.71, 0.03],
 [0.01, 0.12, 0.  , 0.08, 0.  , 0.  , 0.12, 0.02, 0.12, 0.54]]

df_cm = pd.DataFrame(p_array, genres,
                  genres)
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size

ax=sn.heatmap(df_cm, annot=True,annot_kws={"size": 16}, cmap="BuGn")# font size